### 볼린저밴드 
1. 이동평균선 : 데이터 20개의 평균값
2. 상단 밴드 : 이동평균선 + (2 * 데이터 20개의 표준편차)
3. 하단 밴드 : 이동평균선 - (2 * 데이터 20개의 표준편차)
4. 매수의 조건식 : 가격이 하단밴드보다 낮은 경우
5. 매도의 조건식 : 가격이 상단밴드보다 높은 경우

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
df = pd.read_csv("../../csv/AAPL.csv")

In [ ]:
if 'Date' in df.columns:
    df.set_index('Date', inplace=True)

In [ ]:
# 결측치, 무한대 제거 
flag = df.isin([np.nan, np.inf, -np.inf]).any(axis=1)

In [ ]:
df = df.loc[~flag, ['Adj Close']]

In [ ]:
df

In [ ]:
# index를 시계열로 변경 
df.index = pd.to_datetime(df.index)

In [ ]:
# Adj Close의 데이터에서 이동 평균선 구하기 
df.iloc[0:20, 0].mean()
# df.iloc[19 ]위치에 평균 값을 대입

In [ ]:
# 이동평균선 컬럼을 생성하여 결측치를 넣어둔다. 
df['center'] = np.nan

In [ ]:
df.iloc[19, 1] = df.iloc[0:20, 0].mean()

In [ ]:
df.iloc[18:22]

In [ ]:
for i in range(20, len(df), 1):
    df.iloc[i-1, 1] = df.iloc[ i-20 : i, 0 ].mean()

In [ ]:
df.iloc[18:22]

In [ ]:
# rolling(n) : n개의 데이터를 그룹화 
df['center2'] = df['Adj Close'].rolling(20).mean() 

In [ ]:
df.iloc[18:25, ]

In [ ]:
# 상단 밴드를 생성 :  이동평균선 + (2 * 20개 데이터의 표준편차)
df['ub'] = df['center'] + \
    (2 * df['Adj Close'].rolling(20).std())

In [ ]:
# 하단 밴드를 생성 : 이동평균선 - (2 * 20개 데이터의 표준편차)
df['lb'] = df['center'] - \
    (2 * df['Adj Close'].rolling(20).std())

In [ ]:
plt.figure(figsize=(20, 8))
test = df.tail(400)
plt.plot(test['ub'], label='UB')
plt.plot(test['lb'], label='LB')
plt.plot(test['Adj Close'], label='Close')

plt.legend()
plt.show()

In [ ]:
# 투자의 시작 시간 설정해서 데이터필터링
start = '2010-01-01'

In [ ]:
price_df = df.loc[start:, ]

In [ ]:
# 구매 상태 컬럼을 생성 
price_df['trade'] = ""

### 보유 내역 추가 
- 조건식
    - 상단밴드보다 수정 종가가 높거나 같은 경우
        - 현재 보유 상태라면
            - 매도 ( trade = "" )
        - 현재 보유 상태가 아니라면
            - 유지 ( trade = "" )
    - 상단밴드보다는 낮고 하단 밴드보다 수정종가가 높은 경우
        - 현재 보유 상태라면
            - 유지 ( trade = "buy" )
        - 현재 보유 상태가 아니라면
            - 유지 ( trade = "" )
    - 하단밴드보다 수정 종가가 낮거나 같은 경우
        - 현재 보유 상태라면
            - 유지 ( trade = "buy" )
        - 현재 보유 상태가 아니라면
            - 매수 ( trade = "buy" )

In [ ]:
for idx in price_df.index:
    # idx에 들어오는 데이터는? -> 시계열 데이터
    # 수정 종가가 상단의 밴드보다 높거나 같은 경우 
    if price_df.loc[idx,'Adj Close'] >= price_df.loc[idx, 'ub']:
        # 현재 보유 상태라면? 
        if price_df.shift().loc[idx, 'trade'] == 'buy':
            # 오늘의 trade 매도
            price_df.loc[idx, 'trade'] = ""
        else:
            # 오늘의 trade는 유지
            price_df.loc[idx, 'trade'] = ""
    # 수정 종가가 하단의 밴드보다 낮거나 같은 경우 
    elif price_df.loc[idx, 'Adj Close'] <= price_df.loc[idx, 'lb']:
        # 현재 보유 상태라면
        if price_df.shift().loc[idx, 'trade'] == 'buy':
            # 오늘의 trade 유지
            price_df.loc[idx, 'trade'] = 'buy'
        else:
            # 오늘의 trade 매수
            price_df.loc[idx, 'trade'] = 'buy'
    # 수정 종가가 밴드의 사이에 존재하고 있을때
    # elif (price_df.loc[idx, 'Adj Close'] < price_df.loc[idx, 'ub']) \
    #     & \
    #         (price_df.loc[idx, 'Adj Close'] > price_df.loc[idx, 'lb'])
    else:
        # 현재 보유상태라면 
        if price_df.shift().loc[idx, 'trade'] == 'buy':
            # 오늘의 trade 유지 
            price_df.loc[idx, 'trade'] = 'buy'
        else:
            # 오늘의 trade 유지 
            price_df.loc[idx, 'trade'] = ""

In [ ]:
price_df['trade'].value_counts()

### 수익율 계산
- 매수한 날의 수정 종가 
    - 전날의 trade가 ""이고 오늘의 trade가 'buy'인 날의 수정 종가
- 매도한 날의 수정종가 
    - 전날의 trade가 "buy"이고 오늘의 trade가 ""인 날의 수정 종가
- 수익율 
    - 매도한 날의 수정종가 / 매수한 날의 수정종가

In [ ]:
# 수익율 컬럼을 생성 -> 기본값은 1로 대입 
price_df['rtn'] = 1

for idx in price_df.index:
    # 매수가 생성 
    if (price_df.shift().loc[idx, 'trade'] == "") & \
        (price_df.loc[idx, 'trade'] == "buy"):
        # 매수가를 변수에 저장 
        buy = price_df.loc[idx, 'Adj Close']
        print(f"매수일 : {idx}, 매수가 : {buy}")
    # 매도가 생성
    elif (price_df.shift().loc[idx, 'trade'] == "buy") & \
        (price_df.loc[idx, 'trade'] == ""):
        # 매도가를 변수에 저장 
        sell = price_df.loc[idx, 'Adj Close']
        # 수익율 계산
        rtn = sell / buy
        price_df.loc[idx, 'rtn'] = rtn
        print(f"매도일  {idx}, 매도가 : {sell}, 수익율 : {rtn}")


In [ ]:
price_df['acc_rtn'] = price_df['rtn'].cumprod()

In [ ]:
price_df.iloc[-1, -1]

- 밴드 생성 함수 (create_band)
    - 매개변수 5개
        - 데이터프레임 (_df) : 필수
        - 투자의 시작시간(_start) : 2010-01-01 기본값
        - 투자의 종료시간(_end) : 현재 시간 기본값
        - 기준이 되는 컬럼의 이름(_col) : Adj Close 기본값
        - 평균선의 일자수(_cnt) : 20 기본값
    1. _df의 복사본을 생성 (df 변수 생성)
    2. 컬럼에 Date가 존재한다면 Date 컬럼을 인덱스로 변경 
    3. 기준이 되는 컬럼을 제외하고 나머지 컬럼은 모두 제거 
    4. 결측치, 무한대 데이터를 제거 
    5. 이동 평균선, 상단 밴드, 하단 밴드 생성 
    6. 시작 시간, 종료시간은 기준으로 데이터 필터링 
    7. 변경이 된 데이터프레임을 되돌려준다. 

In [ ]:
def create_band(
    _df, 
    _start = '2010-01-01', 
    _end = datetime.now(),
    _col = 'Adj Close', 
    _cnt = 20
):
    # 복사본을 생성 
    df = _df.copy()
    # Date 컬럼에 존재하면 인덱스로 변환
    if 'Date' in df.columns:
        df.set_index('Date', inplace=True)
    # 인덱스를 시계열 데이터로 변경 
    df.index = pd.to_datetime(df.index)
    # 기준이 되는 컬럼을 제외하고 모두 제거 -> 특정 컬럼만 선택
    df = df[[_col]]
    # 결측치, 무한대 제거 
    flag = df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    df = df.loc[~flag, ]
    df['center'] = df[_col].rolling(_cnt).mean()
    std_data = df[_col].rolling(_cnt).std()
    df['ub'] = df['center'] + (2 * std_data)
    df['lb'] = df['center'] - (2 * std_data)
    # 시작 시간과 종료시간으로 데이터를 필터링
    df = df.loc[_start : _end, ]
    return df


In [ ]:
df2 = pd.read_csv('../../csv/AMZN.csv')

In [ ]:
price_df2 = create_band(df2)

- 트레이드 컬럼을 생성하는 함수 
    - 매개변수 1개 
        - create_band() 함수에서 나온 결과 데이터프레임
    1. 데이터프레임의 복사본 생성
    2. trade 컬럼을 생성하여 ""  대입
    3. 밴드의 값들과 기준이 되는 컬럼을 가지고 보유 내역을 생성
    4. 결과를 되돌려준다. 

In [ ]:
def create_trade(_df):
    # 복사본을 생성
    df = _df.copy()
    # trade 컬럼을 생성
    df['trade'] = ""
    # 기준이 되는 컬럼의 이름을 변수에 저장 
    col = df.columns[0]
    # 거래 내역 추가하는 반복문을 사용 
    for idx in df.index:
        # 상단 밴드보다 기준이되는 컬럼의 값이 크거나 같은 경우 
        if df.loc[idx, col] >= df.loc[idx, 'ub']:
            df.loc[idx, 'trade'] = ""
        # 하단 밴드보다 기준이 되는 컬럼의 값이 작거나 같은 경우
        elif df.loc[idx, col] <= df.loc[idx, 'lb']:
            df.loc[idx, 'trade'] = 'buy'
        # 밴드 사이에 기준이 되는 컬럼의 값이 존재하는 경우
        else:
            # 현재 보유중 -> 보유 유지
            # 보유 상태가 아니면 -> 유지
            # 전날의 trade를 그대로 유지 
            df.loc[idx, 'trade'] = df.shift().loc[idx, 'trade']
    return df

In [ ]:
trade_df = create_trade(price_df2)

In [ ]:
trade_df['trade'].value_counts()

- 수익율 계산 함수 생성 
    - 매개변수 1개
        - create_trade() 함수의 결과 데이터프레임
    1. 데이터프레임 복사본 생성 
    2. 기준이 되는 컬럼의 이름을 변수에 저장 
    3. rtn 컬럼을 생성해서 1 대입 
    4. 반복문을 이용하여 수익율 계산 
    5. acc_rtn 컬럼을 생성하여 누적 수익율 계산한 값을 대입 
    6. 데이터프레임과 최종 누적 수익율 데이터를 되돌려준다. 

In [ ]:
def create_rtn(_df):
    # 복사본 생성 
    df = _df.copy()
    # 기준이 되는 컬럼의 이름은 변수에 저장 
    col = df.columns[0]

    # rtn 컬럼을 생성하여 1을 대입 
    df['rtn'] = 1

    # 수익율 계산
    for idx in df.index:
        # 매수 
        if (df.shift().loc[idx, 'trade'] == "") & \
            (df.loc[idx, 'trade'] == 'buy'):
            buy = df.loc[idx, col]
            print(f"매수일 : {idx}, 매수가 : {buy}")
        # 매도
        elif (df.shift().loc[idx, 'trade'] == "buy") & \
            (df.loc[idx, 'trade'] == ""):
            sell = df.loc[idx, col]
            rtn = sell / buy
            df.loc[idx, 'rtn'] = rtn
            print(f"매도일 : {idx}, 매도가 : {sell}, 수익율 : {rtn}")
    # 누적수익율 계산
    df['acc_rtn'] = df['rtn'].cumprod()
    # 최종 누적 수익율
    acc_rtn = df.iloc[-1, -1]
    return df, acc_rtn

In [ ]:
boll_df, boll_rtn = create_rtn(trade_df)

In [ ]:
boll_df

In [56]:
boll_rtn

3.138061358619031